In [ ]:
import torch
print(torch.__version__)

In [ ]:
from datasets import load_dataset 

datasets = load_dataset("nielsr/XFUN", "xfun.ja")

In [ ]:
labels = datasets['train'].features['labels'].feature.names
labels

In [ ]:
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id

In [ ]:
from transformers import LayoutXLMTokenizer, LayoutXLMProcessor, LayoutLMv2FeatureExtractor

processor = LayoutXLMProcessor.from_pretrained('./test-ner')

In [ ]:
from PIL import Image, ImageDraw, ImageFont

image = Image.open('./infer/ja.val/ja_val_0.jpg').convert("RGB")
image

In [ ]:
encoding = processor(image, return_offsets_mapping=True, return_tensors="pt")
offset_mapping = encoding.pop('offset_mapping')
print(encoding.keys())

In [ ]:
encoding['input_ids'].shape

In [ ]:
from transformers import LayoutLMv2ForTokenClassification

# load the fine-tuned model from the hub
model = LayoutLMv2ForTokenClassification.from_pretrained("./test-ner")

# forward pass
outputs = model(**encoding)
print(outputs.logits.shape)

In [ ]:
def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

predictions = outputs.logits.argmax(-1).squeeze().tolist()
token_boxes = encoding.bbox.squeeze().tolist()

width, height = image.size

In [ ]:
import numpy as np

is_subword = np.array(offset_mapping.squeeze().tolist())[:,0] != 0

true_predictions = [id2label[pred] for idx, pred in enumerate(predictions) if not is_subword[idx]]
true_boxes = [unnormalize_box(box, width, height) for idx, box in enumerate(token_boxes) if not is_subword[idx]]

In [ ]:
print(true_predictions)
print(true_boxes)

In [ ]:
from PIL import ImageDraw

draw = ImageDraw.Draw(image)

font = ImageFont.load_default()

def iob_to_label(label):
    label = label[2:]
    if not label:
      return 'other'
    return label

label2color = {'question':'blue', 'answer':'green', 'header':'orange', 'other':'violet'}

for prediction, box in zip(true_predictions, true_boxes):
    predicted_label = iob_to_label(prediction).lower()
    draw.rectangle(box, outline=label2color[predicted_label])
    draw.text((box[0]+10, box[1]-10), text=predicted_label, fill=label2color[predicted_label], font=font)

image

In [ ]:
def infer(img_path, processor, model, label2color):
    image = Image.open(img_path).convert("RGB")
    encoding = processor(image, return_offsets_mapping=True, return_tensors="pt", truncation=True, max_length=514)
    offset_mapping = encoding.pop('offset_mapping')
    outputs = model(**encoding)
    predictions = outputs.logits.argmax(-1).squeeze().tolist()
    token_boxes = encoding.bbox.squeeze().tolist()

    width, height = image.size
    is_subword = np.array(offset_mapping.squeeze().tolist())[:,0] != 0

    true_predictions = [id2label[pred] for idx, pred in enumerate(predictions) if not is_subword[idx]]
    true_boxes = [unnormalize_box(box, width, height) for idx, box in enumerate(token_boxes) if not is_subword[idx]]
    draw = ImageDraw.Draw(image)

    font = ImageFont.load_default()

    for prediction, box in zip(true_predictions, true_boxes):
        predicted_label = iob_to_label(prediction).lower()
        draw.rectangle(box, outline=label2color[predicted_label])
        draw.text((box[0]+10, box[1]-10), text=predicted_label, fill=label2color[predicted_label], font=font)

    return image

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from pathlib import Path
from tqdm.notebook import tqdm

imgs = [img_path for img_path in Path('./infer/ja.val/').glob('*.jpg')][:50]
outputs = [infer(img_path, processor, model) for img_path in tqdm(imgs)]

In [ ]:
for o in outputs:
    display(o)